## Is this just magic?  What is Numba doing to make code run quickly?

When you add the `jit` decorator (or function call), Numba examines the code in the function and then tries to compile it using the LLVM compiler. LLVM takes Numba's translation of the Python code and compiles it into something like assembly code, which is a set of very low-level and very _fast_ instructions. 

Let's create a small, simple example function to poke around in:

In [ ]:
from numba import jit

In [ ]:
@jit
def add(a, b):
    return a + b

In [ ]:
add(1, 1)

Now that we've run `add` once, it is now _compiled_ and we can check out what's happened behind the scenes.  Use the `inspect_types` method to see how Numba translated the function.

In [ ]:
add.inspect_types()

That is a bit more complicated than our original line -- and in fact, there's a bunch of even more complicated stuff going on behind the scenes, but we won't go into that right now. For now, just recognize that Numba is examining the code we wrote, then translating it into a more complex representation that can be efficiently compiled into a super-fast version.

### However...

This translation business is hard and Numba isn't perfect. If it encounters something that it doesn't understand, then it will still *work*, but it will operate in what is called "Object Mode". This is fine, except "Object Mode" can be really, _really_ slow.  

So what can we do to avoid object mode?

Well, first, there's a list of supported features that Numba understands that you can browse at your leisure (but do this later): http://numba.pydata.org/numba-doc/latest/reference/pysupported.html

## Forcing `nopython` mode

The opposite of the slow "object mode" is called `nopython` mode.  That's a kind of confusing name, but it is what it is. The important thing to remember is that `nopython` mode is when Numba is _fast_, so that's what we want. 

But how do we know what "mode" Numba is using?  

That's a good question. We don't always know, and we can't know ahead of time, but we do have one helper to look out for us. 

If we specify `nopython=True`, then Numba will throw an exception and _fail_ to compile when it can't make a function work in `nopython` mode. Then we can try to rewrite that function until it _can_ compile.

Here's a quick example.  First import `numpy` and the `linalg` module from `scipy`.

In [ ]:
import numpy
from scipy import linalg

Define a random square array:

In [ ]:
a = numpy.random.random((5, 5))

Now write a function to pass that array to the `linalq` QR decomposition method:

In [ ]:
def qr_decomposition(a):
    return linalg.decomp_qr.qr(a)

Now let's try it out:

In [ ]:
qr_decomposition(a)

It works!  Ok, now let's try to `jit` it:

In [ ]:
qr_jit = jit()(qr_decomposition)

In [ ]:
qr_jit(a)

It works! Or did it? What if try to add the `nopython=True` flag?  

(Also, remember how we talked about those super weird second set of parentheses?  Here's where they come in)

In [ ]:
qr_jit = jit(nopython=True)(qr_decomposition)

Prepare for a very long error message...

In [ ]:
qr_jit(a)

### Ack

Yes, that's a very long and intimidating looking error message, but just focus on the last few lines, specifically "Failed at nopython". That's Numba telling us that it has no idea what `scipy.linalg.decomp_qr` is, so it can't try to accelerate it. 

It worked the first time because it was in "object" mode but we just asked Numba to _force_ `nopython` mode and it tried (and failed). 

`nopython` mode is so useful, that people got tired of typing it out all of the time, so there's a shortcut!

In [ ]:
from numba import njit

`njit` is exactly the same as `jit` but it always forces `nopython=True`.  And like `jit`, you can use it in a function call, or as a decorator.  Let's try it out on the simple `add` function we started out with:

In [ ]:
def add(a, b):
    return a + b

#### Function call:

In [ ]:
add_jit = njit(add)  # no extra parentheses needed with `njit`

In [ ]:
add_jit(3, 4)

#### Decorator:

In [ ]:
@njit
def add(a, b):
    return a + b

In [ ]:
add(4, 6)

And that's it! 

Unless you have very specific requirements, we recommend always using `njit` over `jit`, so you can guarantee that you're taking advantage of all of Numba's speedup power.